# Population's mental health status detection on a Twitter (X) level. Artem Shtefan

### Imports

In [1]:
import pandas as pd 
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
import gensim
from gensim.models import CoherenceModel, LdaModel
from gensim.corpora.dictionary import Dictionary
!pip install pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis 
!pip install bertopic
from bertopic import BERTopic

/Users/artsh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package stopwords to /Users/artsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/artsh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


2024-01-23 11:55:06.946047: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/artsh/anaconda3/lib/python3.11/site-packages/plotly/express/imshow_utils.py:24: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/Users/artsh/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/artsh/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVe

In [2]:
tweets_df = pd.read_csv('Mental-Health-Twitter.csv')
tweets_df.head()

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1


In [3]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    20000 non-null  int64 
 1   post_id       20000 non-null  int64 
 2   post_created  20000 non-null  object
 3   post_text     20000 non-null  object
 4   user_id       20000 non-null  int64 
 5   followers     20000 non-null  int64 
 6   friends       20000 non-null  int64 
 7   favourites    20000 non-null  int64 
 8   statuses      20000 non-null  int64 
 9   retweets      20000 non-null  int64 
 10  label         20000 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 1.7+ MB


In [4]:
tweets_df.isnull().sum()

Unnamed: 0      0
post_id         0
post_created    0
post_text       0
user_id         0
followers       0
friends         0
favourites      0
statuses        0
retweets        0
label           0
dtype: int64

In [5]:
len(tweets_df)

20000

### Text preprocessing

In [6]:
tweets = tweets_df['post_text']
tweets

0        It's just over 2 years since I was diagnosed w...
1        It's Sunday, I need a break, so I'm planning t...
2        Awake but tired. I need to sleep but my brain ...
3        RT @SewHQ: #Retro bears make perfect gifts and...
4        It’s hard to say whether packing lists are mak...
                               ...                        
19995                A day without sunshine is like night.
19996    Boren's Laws: (1) When in charge, ponder. (2) ...
19997    The flow chart is a most thoroughly oversold p...
19998    Ships are safe in harbor, but they were never ...
19999       Black holes are where God is dividing by zero.
Name: post_text, Length: 20000, dtype: object

In [7]:
tweets_bert = tweets_df['post_text']
tweets_bert

0        It's just over 2 years since I was diagnosed w...
1        It's Sunday, I need a break, so I'm planning t...
2        Awake but tired. I need to sleep but my brain ...
3        RT @SewHQ: #Retro bears make perfect gifts and...
4        It’s hard to say whether packing lists are mak...
                               ...                        
19995                A day without sunshine is like night.
19996    Boren's Laws: (1) When in charge, ponder. (2) ...
19997    The flow chart is a most thoroughly oversold p...
19998    Ships are safe in harbor, but they were never ...
19999       Black holes are where God is dividing by zero.
Name: post_text, Length: 20000, dtype: object

In [8]:
#to lowercase and del white spaces
tweets = tweets.map(lambda x: x.lower())
tweets = tweets.map(lambda x: x.strip())
tweets = tweets.map(lambda x: " ".join(x.split()))
#del uninformative pieces of text
tweets = tweets.map(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))
tweets = tweets.map(lambda x: re.sub(r'\@\w+|\#','', x))
tweets = tweets.map(lambda x: re.sub(r'[^A-Za-z\s]','', x))
tweets = tweets.map(lambda x: re.sub(r'\b(?:im|dont|one|u|rt|amp|youre|thats|cant)\b','', x))
tweets

0        its just over  years since i was diagnosed wit...
1        its sunday i need a break so  planning to spen...
2        awake but tired i need to sleep but my brain h...
3          retro bears make perfect gifts and are great...
4        its hard to say whether packing lists are maki...
                               ...                        
19995                 a day without sunshine is like night
19996    borens laws  when in charge ponder  when in tr...
19997    the flow chart is a most thoroughly oversold p...
19998    ships are safe in harbor but they were never m...
19999        black holes are where god is dividing by zero
Name: post_text, Length: 20000, dtype: object

In [9]:
#tokenization
tweets = tweets.map(lambda x: word_tokenize(x))
tweets

0        [its, just, over, years, since, i, was, diagno...
1        [its, sunday, i, need, a, break, so, planning,...
2        [awake, but, tired, i, need, to, sleep, but, m...
3        [retro, bears, make, perfect, gifts, and, are,...
4        [its, hard, to, say, whether, packing, lists, ...
                               ...                        
19995         [a, day, without, sunshine, is, like, night]
19996    [borens, laws, when, in, charge, ponder, when,...
19997    [the, flow, chart, is, a, most, thoroughly, ov...
19998    [ships, are, safe, in, harbor, but, they, were...
19999    [black, holes, are, where, god, is, dividing, ...
Name: post_text, Length: 20000, dtype: object

In [10]:
#del stopwords
stops = nltk.corpus.stopwords.words('english')
for i in range(0,len(tweets)):
    tweets[i] = [word for word in tweets[i] if not word in list(stops)]
tweets

0        [years, since, diagnosed, anxiety, depression,...
1        [sunday, need, break, planning, spend, little,...
2                [awake, tired, need, sleep, brain, ideas]
3        [retro, bears, make, perfect, gifts, great, be...
4        [hard, say, whether, packing, lists, making, l...
                               ...                        
19995                [day, without, sunshine, like, night]
19996    [borens, laws, charge, ponder, trouble, delega...
19997    [flow, chart, thoroughly, oversold, piece, pro...
19998            [ships, safe, harbor, never, meant, stay]
19999                  [black, holes, god, dividing, zero]
Name: post_text, Length: 20000, dtype: object

In [11]:
#del uninformative


In [12]:
#lemmatization
lemmatizer = WordNetLemmatizer()
for i in range(0,len(tweets)):
    words = []
    for word in tweets[i]:
        words.append(lemmatizer.lemmatize(word))
    tweets[i] = words
tweets

0        [year, since, diagnosed, anxiety, depression, ...
1        [sunday, need, break, planning, spend, little,...
2                 [awake, tired, need, sleep, brain, idea]
3        [retro, bear, make, perfect, gift, great, begi...
4        [hard, say, whether, packing, list, making, li...
                               ...                        
19995                [day, without, sunshine, like, night]
19996    [borens, law, charge, ponder, trouble, delegat...
19997    [flow, chart, thoroughly, oversold, piece, pro...
19998             [ship, safe, harbor, never, meant, stay]
19999                   [black, hole, god, dividing, zero]
Name: post_text, Length: 20000, dtype: object

### LDA model creating

In [13]:
#optimal number of topics
dictionary = Dictionary(tweets)
tweets_corpus = [dictionary.doc2bow(tweet) for tweet in tweets]
for i in range (2,80):
    lda_model = LdaModel(tweets_corpus, num_topics=i, id2word=dictionary)
    coherence_lda_model = CoherenceModel(model=lda_model, texts=tweets, dictionary=dictionary)
    coherence_lda = coherence_lda_model.get_coherence()
    print(f'Num Topics = {i} has Coherence Value of {coherence_lda}')

Num Topics = 2 has Coherence Value of 0.3255296350062428
Num Topics = 3 has Coherence Value of 0.36160063433201106
Num Topics = 4 has Coherence Value of 0.3363324194619777
Num Topics = 5 has Coherence Value of 0.3879000903723475
Num Topics = 6 has Coherence Value of 0.374426096436669
Num Topics = 7 has Coherence Value of 0.40890855447270197
Num Topics = 8 has Coherence Value of 0.4125382160035826
Num Topics = 9 has Coherence Value of 0.384677010725766
Num Topics = 10 has Coherence Value of 0.4161385906881625
Num Topics = 11 has Coherence Value of 0.3847604099859914
Num Topics = 12 has Coherence Value of 0.3951449954599448
Num Topics = 13 has Coherence Value of 0.4369782242087741
Num Topics = 14 has Coherence Value of 0.45053397391352074
Num Topics = 15 has Coherence Value of 0.43946506213019065
Num Topics = 16 has Coherence Value of 0.43452076468200884
Num Topics = 17 has Coherence Value of 0.46352488940306813
Num Topics = 18 has Coherence Value of 0.477887941289658
Num Topics = 19 has

In [14]:
#building LDA model
best_lda = LdaModel(corpus=tweets_corpus, id2word=dictionary, num_topics=54, passes=5)

In [15]:
for idx, topic in best_lda.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))

Topic: 0 
Words: 0.067*"wont" + 0.053*"else" + 0.052*"moving" + 0.041*"medium" + 0.038*"either" + 0.029*"air" + 0.024*"football" + 0.022*"autism" + 0.021*"monday" + 0.018*"ht"
Topic: 1 
Words: 0.158*"well" + 0.099*"ok" + 0.068*"as" + 0.058*"law" + 0.026*"photo" + 0.023*"space" + 0.021*"fall" + 0.019*"yesterday" + 0.018*"solution" + 0.018*"opinion"
Topic: 2 
Words: 0.174*"got" + 0.091*"lol" + 0.058*"lie" + 0.052*"hour" + 0.050*"cool" + 0.042*"left" + 0.041*"cat" + 0.030*"called" + 0.030*"michael" + 0.027*"hurt"
Topic: 3 
Words: 0.204*"life" + 0.124*"actually" + 0.090*"mean" + 0.041*"kill" + 0.037*"aint" + 0.036*"okay" + 0.028*"ready" + 0.021*"glad" + 0.020*"happiness" + 0.019*"mate"
Topic: 4 
Words: 0.158*"real" + 0.084*"everyone" + 0.065*"mom" + 0.063*"must" + 0.054*"face" + 0.042*"havent" + 0.027*"mad" + 0.023*"decided" + 0.021*"choice" + 0.018*"watched"
Topic: 5 
Words: 0.122*"he" + 0.085*"white" + 0.068*"light" + 0.044*"obama" + 0.038*"v" + 0.032*"study" + 0.026*"doctor" + 0.024*"se

In [25]:
vis_data = gensimvis.prepare(best_lda, tweets_corpus, dictionary)
pyLDAvis.save_html(vis_data, 'pyLDAvis.html')
pyLDAvis.display(vis_data)

/Users/artsh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### BERTopic

In [17]:
bert = BERTopic(top_n_words=30, calculate_probabilities=True, embedding_model='all-MiniLM-L6-v2')
bertopic, probs = bert.fit_transform(tweets_bert)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [36]:
bert.visualize_topics().write_html('bert_top_vis.html')

In [35]:
bert.visualize_barchart(topics=[143, 46, 0, 18, 178, 14, 235, 7, 36, 61, 106, 137])